In [0]:
%pip install databricks-sdk -U -qqqq backoff databricks-openai uv databricks-agents mlflow-skinny[databricks]

dbutils.library.restartPython() # Restart Python after install

In [0]:
import requests
import xml.etree.ElementTree as ET
import json
import warnings
from typing import Any, Callable, Generator, Optional
from uuid import uuid4
import backoff
import mlflow
import openai
from databricks.sdk import WorkspaceClient
from databricks_openai import UCFunctionToolkit, VectorSearchRetrieverTool
from mlflow.entities import SpanType
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
    output_to_responses_items_stream,
    to_chat_completions_input)
from openai import OpenAI
from pydantic import BaseModel
from unitycatalog.ai.core.base import get_uc_function_client

In [0]:
# create secret scope
w = WorkspaceClient()
w.secrets.create_scope(scope="elm")

In [0]:
print("Available scopes:")
scopes = dbutils.secrets.listScopes()
for scope in scopes:
    print(f"- {scope.name}")

### Put sandbox credentials into Databricks

In [0]:
#Saves eBay credentials (dev_id, app_id, cert_id, user_token) into Databricks secret scope elm.
w.secrets.put_secret("elm",
    "production_dev_id",
    string_value="11111")
w.secrets.put_secret("elm",
    "production_app_id",
    string_value="22222")
w.secrets.put_secret("elm",
    "production_cert_id",
    string_value="33333")
w.secrets.put_secret("elm",
    "production_user_token",
    string_value="44444")

elm={"production_dev_id":"...", "production_app_id":"", "production_cert_id":"", "production_user_token":""}

In [0]:
#read secret scopes
my_token_prod = dbutils.secrets.get(scope="elm", key="production_user_token")
print(my_token_prod)

In [0]:
# Call eBay Trading API (GetMyeBaySelling) in production using stored secrets to retrieve my active listings
#ENDPOINT = "https://api.sandbox.ebay.com/ws/api.dll"
ENDPOINT = "https://api.ebay.com/ws/api.dll" # Using the production eBay Trading API (not sandbox).
# These headers tell eBay which operation we are calling and identify my app.
headers = {
    "X-EBAY-API-CALL-NAME": "GetMyeBaySelling",
    "X-EBAY-API-SITEID": "0",              # 0 = US site
    "X-EBAY-API-COMPATIBILITY-LEVEL": "967",  # example; check docs for current level
    "X-EBAY-API-DEV-NAME": dbutils.secrets.get(scope="elm", key="production_dev_id").strip(),
    "X-EBAY-API-APP-NAME": dbutils.secrets.get(scope="elm", key="production_app_id").strip(),
    "X-EBAY-API-CERT-NAME": dbutils.secrets.get(scope="elm", key="production_cert_id").strip(),
    "Content-Type": "text/xml"
}

# CALL-NAME: GetMyeBaySelling, GetUser, 

# build an XML request for eBay
body = f"""<?xml version="1.0" encoding="utf-8"?>
<GetMyeBaySellingRequest xmlns="urn:ebay:apis:eBLBaseComponents">
  <RequesterCredentials>
    <eBayAuthToken>{my_token_prod.strip()}</eBayAuthToken> 
  </RequesterCredentials>
  <DetailLevel>ReturnAll</DetailLevel>
</GetMyeBaySellingRequest>
"""
# trading_user_token
# Sending the HTTP request
resp = requests.post(ENDPOINT, data=body, headers=headers)
print(resp.status_code)# Quick check: did the request succeed at the HTTP level?
print(resp.text)# Prints the raw XML response from eBay

In [0]:
print(requests.get("https://www.google.com").status_code)

In [0]:
# Sends Trading API request to eBay.
ENDPOINT = "https://api.ebay.com/ws/api.dll"

# Using dbutils.secrets.get means we don’t hard-code secrets in the notebook.
DEV_ID  = dbutils.secrets.get("elm", "production_dev_id")
APP_ID  = dbutils.secrets.get("elm", "production_app_id")
CERT_ID = dbutils.secrets.get("elm", "production_cert_id")
USER_TOKEN = my_token_prod  # my prod user token string

# read my eBay application credentials from Databricks secrets...
def call_trading_api(call_name: str, body_xml: str) -> str:
    headers = {
        # Builds the HTTP headers eBay expects
        "X-EBAY-API-CALL-NAME": call_name,
        "X-EBAY-API-SITEID": "0",                 # US
        "X-EBAY-API-COMPATIBILITY-LEVEL": "967",  # ok for testing
        "X-EBAY-API-DEV-NAME": DEV_ID,
        "X-EBAY-API-APP-NAME": APP_ID,
        "X-EBAY-API-CERT-NAME": CERT_ID,
        "X-EBAY-API-REQUEST-ENCODING": "XML",
        "Content-Type": "text/xml",
    }
    # Sends the HTTP POST request to eBay
    resp = requests.post(ENDPOINT, data=body_xml, headers=headers)
    resp.raise_for_status()# Error handling
    return resp.text# Returns the raw XML response as a string.

In [0]:
# Get a list of active items (ItemID + Title)
def list_active_items():
    body = f"""<?xml version="1.0" encoding="utf-8"?>
<GetMyeBaySellingRequest xmlns="urn:ebay:apis:eBLBaseComponents">
  <RequesterCredentials>
    <eBayAuthToken>{USER_TOKEN}</eBayAuthToken>
  </RequesterCredentials>
  <ActiveList>
    <Include>true</Include>
  </ActiveList>
</GetMyeBaySellingRequest>
"""
# parses the XML response
    xml_str = call_trading_api("GetMyeBaySelling", body)
    ns = {"e": "urn:ebay:apis:eBLBaseComponents"}
    root = ET.fromstring(xml_str)

    for item in root.findall(".//e:ActiveList/e:ItemArray/e:Item", ns):
        item_id = item.findtext("e:ItemID", default="", namespaces=ns)
        title   = item.findtext("e:Title",  default="", namespaces=ns)
        print(item_id, " | ", title)

# run this once to see my ItemIDs
list_active_items()

In [0]:
# Get description for a specific listing
def get_item_description(item_id: str) -> str:
    body = f"""<?xml version="1.0" encoding="utf-8"?>
<GetItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
  <RequesterCredentials>
    <eBayAuthToken>{USER_TOKEN}</eBayAuthToken>
  </RequesterCredentials>
  <ItemID>{item_id}</ItemID>
  <DetailLevel>ReturnAll</DetailLevel>
</GetItemRequest>
"""
    xml_str = call_trading_api("GetItem", body)
    ns = {"e": "urn:ebay:apis:eBLBaseComponents"}
    root = ET.fromstring(xml_str)
    description = root.findtext(".//e:Item/e:Description", default="", namespaces=ns)
    return description

# example usage:
item_id = "256077000122"
print(get_item_description(item_id))

In [0]:
# Call eBay Trading API (GetMyeBaySelling) in production using stored secrets to retrieve my active listings
#ENDPOINT = "https://api.sandbox.ebay.com/ws/api.dll"
ENDPOINT = "https://api.ebay.com/ws/api.dll"
headers = {
    "X-EBAY-API-CALL-NAME": "GetOrders",
    "X-EBAY-API-SITEID": "0",              # 0 = US site
    "X-EBAY-API-COMPATIBILITY-LEVEL": "967",  # example; check docs for current level
    "X-EBAY-API-DEV-NAME": dbutils.secrets.get(scope="elm", key="production_dev_id").strip(),
    "X-EBAY-API-APP-NAME": dbutils.secrets.get(scope="elm", key="production_app_id").strip(),
    "X-EBAY-API-CERT-NAME": dbutils.secrets.get(scope="elm", key="production_cert_id").strip(),
    "Content-Type": "text/xml"
}

# CALL-NAME: GetOrders, GetUser, GetMyeBaySelling, 


body = f"""<?xml version="1.0" encoding="utf-8"?>
<GetOrdersRequest xmlns="urn:ebay:apis:eBLBaseComponents">
  <RequesterCredentials>
    <eBayAuthToken>{my_token_prod.strip()}</eBayAuthToken>
  </RequesterCredentials>
  <DetailLevel>ReturnAll</DetailLevel>
</GetOrdersRequest>
"""
# production _user_token
resp = requests.post(ENDPOINT, data=body, headers=headers)
print(resp.status_code)
print(resp.text)

In [0]:
# Generic Trading API caller
DEV_ID  = dbutils.secrets.get("elm", "production_dev_id").strip()
APP_ID  = dbutils.secrets.get("elm", "production_app_id").strip()
CERT_ID = dbutils.secrets.get("elm", "production_cert_id").strip()
USER_TOKEN = my_token_prod.strip()

# Builds headers eBay requires
def call_trading_api(call_name: str, body_xml: str) -> str:
    headers = {
        "X-EBAY-API-CALL-NAME": call_name,
        "X-EBAY-API-SITEID": "0",                 # US
        "X-EBAY-API-COMPATIBILITY-LEVEL": "967",  # keep for now
        "X-EBAY-API-DEV-NAME": DEV_ID,
        "X-EBAY-API-APP-NAME": APP_ID,
        "X-EBAY-API-CERT-NAME": CERT_ID,
        "X-EBAY-API-REQUEST-ENCODING": "XML",
        "Content-Type": "text/xml",
    }
    resp = requests.post(ENDPOINT, data=body_xml, headers=headers)# Sends the HTTP POST
    resp.raise_for_status()
    return resp.text # Returns the raw XML response

In [0]:
# Calling eBay’s GetMyeBaySelling API to Retrieve Selling Activity
body_2 = f"""<?xml version="1.0" encoding="utf-8"?>
<GetMyeBaySellingRequest xmlns="urn:ebay:apis:eBLBaseComponents">
  <RequesterCredentials>
    <eBayAuthToken>{my_token_prod.strip()}</eBayAuthToken>
  </RequesterCredentials>
  <DetailLevel>ReturnAll</DetailLevel>
</GetMyeBaySellingRequest>
"""
print(call_trading_api("GetMyeBaySelling", body_2)) #Call the generic Trading API helper

In [0]:
# Sends a ReviseItem request to update the description on eBay.
def revise_item_description(item_id: str, new_description: str) -> str:
    """
    Update the description HTML of a single listing.
    Returns the raw XML response.
    """
# Building the ReviseItem XML
    body = f"""<?xml version="1.0" encoding="utf-8"?>
          <ReviseItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
            <RequesterCredentials>
              <eBayAuthToken>{USER_TOKEN}</eBayAuthToken>
            </RequesterCredentials>
            <Item>
              <ItemID>{item_id}</ItemID>
              <Description><![CDATA[
          {new_description}
              ]]></Description>
            </Item>
          </ReviseItemRequest>
          """
    xml_str = call_trading_api("ReviseItem", body)# Sending to eBay
    return xml_str

In [0]:
# executing a live update on eBay listing
print(revise_item_description("256077000122", """Topcon 9003A Robotic Total Station with RC-3R
No battery is included.
Little dent on case, please see the pictures for unit condition.
What you see in the pictures is exactly what you will receive.&&&"""))

In [0]:
def get_item_description_tool(item_id: str) -> dict:
    """
    Agent tool: fetch the current description of an eBay listing.
    Returns a dict with item_id and description.
    """
    # Build the GetItem XML request
    body = f"""<?xml version="1.0" encoding="utf-8"?>
<GetItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
  <RequesterCredentials>
    <eBayAuthToken>{USER_TOKEN}</eBayAuthToken>
  </RequesterCredentials>
  <ItemID>{item_id}</ItemID>
  <DetailLevel>ReturnAll</DetailLevel>
</GetItemRequest>
"""
# Call the Trading API
    xml_str = call_trading_api("GetItem", body)
    # Parse the XML to extract the description
    ns = {"e": "urn:ebay:apis:eBLBaseComponents"}
    root = ET.fromstring(xml_str)
    desc = root.findtext(".//e:Item/e:Description", default="", namespaces=ns)
    return {"item_id": item_id, "description": desc} # Return tool-friendly dict

In [0]:
#“Tool Wrapper: Revise eBay Listing Description and Return Ack Status”
# Function signature & docstring
def tool_revise_description(item_id: str, new_description: str) -> dict:
    """
    Tool: revise the eBay listing description for a given item_id.
    Returns a small JSON summary.
    """
    # Call the low-level Trading API helper
    xml_resp = revise_item_description(item_id, new_description) # string (xml)
    # helper function 2
    # helper function 3
    # ....
    # simple success check
    # Parse eBay’s response
    ns = {"e": "urn:ebay:apis:eBLBaseComponents"}
    root = ET.fromstring(xml_resp)
    ack = root.findtext("e:Ack", default="Unknown", namespaces=ns)
    # Return a JSON-style summary
    return {
        "ack": ack,
        "item_id": item_id,
        "new_description_sample": new_description,
    }


In [0]:
# Define my LLM endpoint and system prompt
LLM_ENDPOINT_NAME = "databricks-gpt-oss-120b"
SYSTEM_PROMPT = """
You are Ellie's eBay listing assistant.

Tools:
- get_item_description(item_id): fetches the current description of a listing.
- revise_item_description(item_id, new_description): updates the listing description on eBay.

Behavior:
- If the user asks for help improving a description, first call get_item_description, propose a clearer version, and ask for confirmation.
- However, if the user explicitly provides BOTH an item_id and the full new description text and clearly says they want to apply it
  (e.g. "apply this now", "I confirm, update the listing"), then you may call revise_item_description directly without asking again.
- Never modify multiple items at once unless the user explicitly lists all ItemIDs and confirms it.
"""
## Define tools for your agent, enabling it to retrieve data or take actions
class ToolInfo(BaseModel):
    name: str
    spec: dict
    exec_fn: Callable
# ---- Register tools -------------------------------------------------
TOOL_INFOS: list[ToolInfo] = []
# 1) Spec for your ebay tool (OpenAI-style function schema)
revise_desc_tool_spec = {"type": "function",
    "function": {
        "name": "revise_item_description",
        "description": (
            "Update the description of a single eBay listing. "
            "Use only after the user has confirmed the new wording and provided an exact item_id."),
        "parameters": {
            "type": "object",
            "properties": {
                "item_id": {
                    "type": "string",
                    "description": "The eBay ItemID of the listing to update."},
                "new_description": {
                    "type": "string",
                    "description": "The new HTML or plain-text description to set on the listing."
                }},
            "required": ["item_id", "new_description"]}}}

# 2) Add my tool to TOOL_INFOS
TOOL_INFOS.append(ToolInfo(
        name=revise_desc_tool_spec["function"]["name"],
        spec=revise_desc_tool_spec,
        exec_fn=tool_revise_description   # my Python function
    ))

get_desc_tool_spec = {"type": "function",
    "function": {
        "name": "get_item_description",
        "description": "Fetch the current description of an eBay listing by ItemID.",
        "parameters": {
            "type": "object",
            "properties": {
                "item_id": {
                    "type": "string",
                    "description": "The eBay ItemID of the listing."
                }
            },
            "required": ["item_id"]}}}

TOOL_INFOS.append(
    ToolInfo(
        name=get_desc_tool_spec["function"]["name"],
        spec=get_desc_tool_spec,
        exec_fn=get_item_description_tool,
    )
)

# This class is where my interface Databricks LLMs, tools, and MLflow together.
class ToolCallingAgent(ResponsesAgent):
    """
    Class representing a tool-calling Agent.
    Handles both tool execution via exec_fn and LLM interactions via model serving.
    """

    def __init__(self, llm_endpoint: str, tools: list[ToolInfo]):
        """Initializes the ToolCallingAgent with tools."""
        self.llm_endpoint = llm_endpoint
        self.workspace_client = WorkspaceClient()
        self.model_serving_client: OpenAI = (
            self.workspace_client.serving_endpoints.get_open_ai_client()
        )
        self._tools_dict = {tool.name: tool for tool in tools}

# Tool specs and execution:
    def get_tool_specs(self) -> list[dict]:
        """Returns tool specifications in the format OpenAI expects."""
        return [tool_info.spec for tool_info in self._tools_dict.values()]

    @mlflow.trace(span_type=SpanType.TOOL)
    def execute_tool(self, tool_name: str, args: dict) -> Any:
        """Executes the specified tool with the given arguments."""
        return self._tools_dict[tool_name].exec_fn(**args)
    
# Calling the LLM (with retries + streaming):
    @backoff.on_exception(backoff.expo, openai.RateLimitError)
    @mlflow.trace(span_type=SpanType.LLM)
    def call_llm(self, messages: list[dict[str, Any]]) -> Generator[dict[str, Any], None, None]:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="PydanticSerializationUnexpectedValue")
            for chunk in self.model_serving_client.chat.completions.create(
                model=self.llm_endpoint,
                messages=to_chat_completions_input(messages),
                tools=self.get_tool_specs(),
                stream=True,
            ):
                yield chunk.to_dict()

# Handling a tool call:
    def handle_tool_call(
        self, tool_call: dict[str, Any], messages: list[dict[str, Any]]
    ) -> ResponsesAgentStreamEvent:
        """
        Execute tool calls, add them to the running message history, and return a ResponsesStreamEvent w/ tool output
        """
        args = json.loads(tool_call["arguments"])
        result = str(self.execute_tool(tool_name=tool_call["name"], args=args))

        tool_call_output = self.create_function_call_output_item(tool_call["call_id"], result)
        messages.append(tool_call_output)
        return ResponsesAgentStreamEvent(type="response.output_item.done", item=tool_call_output)

# Main loop: LLM ↔ tools ↔ LLM
    def call_and_run_tools(
        self,
        messages: list[dict[str, Any]],
        max_iter: int = 10,
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        for _ in range(max_iter):
            last_msg = messages[-1]
            if last_msg.get("role", None) == "assistant":
                return
            elif last_msg.get("type", None) == "function_call":
                yield self.handle_tool_call(last_msg, messages)
            else:
                yield from output_to_responses_items_stream(
                    chunks=self.call_llm(messages), aggregator=messages
                )

        yield ResponsesAgentStreamEvent(
            type="response.output_item.done",
            item=self.create_text_output_item("Max iterations reached. Stopping.", str(uuid4())),
        )

# predict collects final items into a ResponsesAgentResponse.
    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done"
        ]
        return ResponsesAgentResponse(output=outputs, custom_outputs=request.custom_inputs)

# predict_stream seeds the messages with your system prompt, then calls the tool loop.
    def predict_stream(
        self, request: ResponsesAgentRequest
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        messages = [{"role": "system", "content": SYSTEM_PROMPT}] + [
            i.model_dump() for i in request.input
        ]
        yield from self.call_and_run_tools(messages=messages)

#####################################################################################

In [0]:
# Registering and using the agent
mlflow.openai.autolog()# Enable MLflow autologging for OpenAI-style calls
# Create an instance of my custom agent
Ebay_Agent = ToolCallingAgent(llm_endpoint=LLM_ENDPOINT_NAME, tools=TOOL_INFOS)
mlflow.models.set_model(Ebay_Agent) # Register this agent as the active MLflow model

In [0]:
# End-to-End Example: Agent Updating an eBay Listing Description on Command
item_id = "256077000122" # Define which listing and what new description to apply
new_desc = """
Topcon 9003A Robotic Total Station with RC-3R\n\n- Unit: Topcon 9003A robotic total station\n- Remote: RC-3R included\n- Battery: No battery included\n- Condition: Used — little dent on carrying case (see photos). What you see in the pictures is exactly what you will receive.\n- Notes: Sold as-is. Please review all images carefully before purchasing...
"""
# Build the natural-language user message
user_msg = f"""
Item ID: {item_id}.
Set the description to exactly the following text and APPLY IT NOW (no further confirmation):

{new_desc}
"""

result = Ebay_Agent.predict({"input": [{"role": "user", "content": user_msg}]})# Call the agent
print(result.model_dump(exclude_none=True))# Inspect the result

In [0]:
# the live eBay description with the text above.
# confirm and apply the change message to my agent, using the same ItemID
item_id = "256077000122"

# Build a short confirmation message
user_msg = (
    f"Item ID: {item_id}. "
    "Yes, update.")

result = Ebay_Agent.predict({"input": [{"role": "user", "content": user_msg}]})# Call the agent
print(result.model_dump(exclude_none=True))# Inspect the structured result